In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datetime import datetime,timedelta
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
user_index = json.load(open("../data/user_index.json",'r'))
commitment_index = json.load(open("../data/commitment_index.json",'r'))

In [3]:
max_user_id = max([int(i) for i in user_index.keys()])

In [4]:
for k,v in tqdm(user_index.items()):
    user_index[k]['date'] = datetime.strptime(v['date'], '%d %b %Y')

  0%|          | 0/521632 [00:00<?, ?it/s]

In [5]:
id_for_1_jan_2023 = 722621
for k,v in tqdm(user_index.items()):
    if int(k)<id_for_1_jan_2023 and v['date'].year == 2023:
        user_index[k]['date'] = None

  0%|          | 0/521632 [00:00<?, ?it/s]

In [6]:
last_date = user_index['1']['date']
for i in range(1, max_user_id+1):
    if str(i) in user_index.keys():
        if user_index[str(i)]['date'] == None:
            user_index[str(i)]['date'] = last_date
        else:
            last_date = user_index[str(i)]['date']

In [7]:
for k,v in tqdm(commitment_index.items()):
    commitment_index[k]['start_date'] = datetime.strptime(v['start_date'], '%B %d, %Y')
    commitment_index[k]['end_date'] = datetime.strptime(v['end_date'], '%B %d, %Y')

  0%|          | 0/319928 [00:00<?, ?it/s]

In [8]:
for k,v in tqdm(commitment_index.items()):
    if v['start_date'].year < 2007:
        deta = v['end_date'] - v['start_date']
        commitment_index[k]['start_date'] = user_index[str(v['user_id'])]['date']
        commitment_index[k]['end_date'] = commitment_index[k]['start_date'] + deta

  0%|          | 0/319928 [00:00<?, ?it/s]

Extract data

In [51]:
raw_data = []
for k,v in commitment_index.items():
    raw_data.append(f"{v['title']}. {v['description']}".lower())
#     raw_data.append(f"{v['title']}".lower())

In [52]:
data = list(set(raw_data))

In [53]:
len(data)

180846

In [54]:
# data = data[:10000]

In [55]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [56]:
corpus_embeddings = model.encode(data, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/2826 [00:00<?, ?it/s]

In [57]:
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.85)

  0%|          | 0/177 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [58]:
len(clusters)

1214

In [59]:
for i, cluster in enumerate(clusters[:100]):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:5]:
        print("\t", data[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-5:]:
        print("\t", data[sentence_id])


Cluster 1, #9558 Elements 
	 lose weight. lose 45 lbs. in 78 week(s)
	 lose weight. lose 42 kgs in 42 week(s)
	 lose weight. lose 13.4 lbs. in 6 week(s)
	 lose weight. lose 18 lbs. in 13 week(s)
	 lose weight. lose 36.5 lbs. in 15 week(s)
	 ...
	 lose weight. lose 4.9 lbs. in 3 week(s)
	 lose weight. lose 38 lbs. in 40 week(s)
	 lose weight. lose 10.1 kgs in 24 week(s)
	 lose weight. lose 30 kgs in 50 week(s)
	 lose weight. lose 18 kgs in 44 week(s)

Cluster 2, #2789 Elements 
	 race!. run and finish the imt marathon marathon (des moines) on may 11, 2014
	 race!. run and finish the miley race (little turtles) on may 23, 2017
	 race!. run and finish the 马拉松 race (杭州) on july 24, 2015
	 race!. run and finish the test marathon (montreal) on january 31, 2018
	 race!. run and finish the boston marathon marathon (boston, ma) on april 18, 2011
	 ...
	 race!. run and finish the jog4joe race (ottumwa, ia) on may 26, 2014
	 race!. run and finish the bjønnaasen opp race (bjønnaasen, raelingen) o